In [10]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import spacy
import torchtext
from torchtext import data 
from pathlib import Path
import pandas as pd
import spacy

In [11]:
torch.__version__

'1.5.1'

In [12]:
torchtext.__version__

'0.5.0'

## Анализ тональности текста

датасет [sentiment140](http://help.sentiment140.com/for-students). Используем training.1600000.processed.noemoticon.csv

In [4]:
tweetsDF = pd.read_csv("input/training.1600000.processed.noemoticon.csv", engine="python", header=None)

In [5]:
tweetsDF.head(5)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
tweetsDF[0].value_counts()

4    800000
0    800000
Name: 0, dtype: int64

In [7]:
tweetsDF["sentiment_cat"] = tweetsDF[0].astype('category')
tweetsDF["sentiment"] = tweetsDF["sentiment_cat"].cat.codes
tweetsDF.to_csv("output/train-processed.csv", header=None, index=None)      
tweetsDF.sample(10000).to_csv("output/train-processed-sample.csv", header=None, index=None)

In [8]:
tweetsDF.head(5)

,0,1,2,3,4,5,sentiment_cat,sentiment
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,0,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,0,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",0,0


torchtext генерирует набор данных из json/csv с определенными типами параметров полей. [Подробнее тут](https://torchtext.readthedocs.io/en/latest/data.html#field)

Нас интересуют только маркировки и текст твитов. Остальные поля мы дропнем. Метку определим как нетекстовое поле. Текст токенизируем с помощью spaCy и переведем в нижний регистр

**Чтобы работать с английским текстом в spacy** потребуетсякачнуть локаль

```
python -m spacy download en
```

In [13]:
LABEL = data.LabelField()
TWEET = data.Field(tokenize='spacy', lower=True)

fields = [('score',None), ('id',None),('date',None),('query',None),
          ('name',None),
          ('tweet', TWEET),('category',None),('label',LABEL)]

При помощи TabularDataset применим определение полей к нашим данным

In [14]:
twitterDataset = torchtext.data.TabularDataset(
    path="output/train-processed-sample.csv",
    format="CSV", 
    fields=fields,
    skip_header=False
)

Посплитим на трейн, валид, тест

In [15]:
train, test, valid = twitterDataset.split(
    split_ratio=[0.6,0.2,0.2],
    stratified=True, 
    strata_field='label'
)

print(len(train), len(test), len(valid))

6000 2000 2000


In [16]:
vars(train.examples[7])

{'tweet': ['loves', 'clean', ',', 'fresh', 'smelling', 'sheets'], 'label': '1'}

#### Построение словаря

с помощью методв build_vocab с параметром max_size создаем словарь, ограничив его наиболее обзщеупотребительными (например 20тыс.) Torchtext добавить еще два токена <unk> для неизвестных слов и <pad> токен отступа, который будет использоваться для подгонки всего текста до одного размера, чтобы обеспечить пакетирование в pytorch. Можно задать и другие доп.токены, но они не включены по умолчанию.

In [17]:
vocab_size = 20000
TWEET.build_vocab(train, max_size = vocab_size)
LABEL.build_vocab(train)

In [18]:
len(TWEET.vocab)

13471

In [19]:
TWEET.vocab.freqs.most_common(10)

[('i', 3798),
 ('!', 3221),
 ('.', 3128),
 (' ', 2187),
 ('to', 2097),
 ('the', 1960),
 (',', 1878),
 ('a', 1471),
 ('my', 1208),
 ('you', 1124)]

Наконец создадим загрузчик с помощью BucketIterator

In [21]:
device = "cuda"
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test),
    batch_size = 32,
    device = device,
    sort_key = lambda x: len(x.tweet),
    sort_within_batch = False)

## Создание модели

В первом слое создаем эмбеддинги - 300-мерный векторы. Затем следует LSTM со 100 скрытыми слоями. На выходе полносвязный слой с тремя классами (отрицательный, положительный или нейтральный твит).

In [22]:
class OurFirstLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(OurFirstLSTM, self).__init__()
    
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(
            input_size=embedding_dim,  
            hidden_size=hidden_size, 
            num_layers=1
        )
        self.predictor = nn.Linear(hidden_size, 2)

    def forward(self, seq):
        output, (hidden,_) = self.encoder(self.embedding(seq))
        preds = self.predictor(hidden.squeeze(0))
        return preds

model = OurFirstLSTM(100,300, 20002)
model.to(device)

OurFirstLSTM(
  (embedding): Embedding(20002, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=2, bias=True)
)

## Обучение

Тут потребуется ссылаться на batch.tweet и batch.label, чтобы получить конкретные поля, которые нас интересуют.

In [23]:
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs + 1):
     
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.tweet.size(0)
        training_loss /= len(train_iterator)
 
        
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            valid_loss += loss.data.item() * batch.tweet.size(0)
 
        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

In [24]:
train(10, model, optimizer, criterion, train_iterator, valid_iterator)

Epoch: 1, Training Loss: 24.67, Validation Loss: 13.07
Epoch: 2, Training Loss: 22.42, Validation Loss: 12.81
Epoch: 3, Training Loss: 20.89, Validation Loss: 14.97
Epoch: 4, Training Loss: 19.48, Validation Loss: 13.68
Epoch: 5, Training Loss: 18.29, Validation Loss: 14.14
Epoch: 6, Training Loss: 19.32, Validation Loss: 15.57
Epoch: 7, Training Loss: 19.93, Validation Loss: 14.05
Epoch: 8, Training Loss: 19.67, Validation Loss: 14.98
Epoch: 9, Training Loss: 19.12, Validation Loss: 16.36
Epoch: 10, Training Loss: 18.85, Validation Loss: 15.11


## Предсказания

Недостаток torchtext - его сложно заставить делать предикшены. В нашем случае мы эмулируем конвеер обработки и делаем прогноз на выходе.

В этой функции мы вызываем preprocess() , который делает токенизацию на основе SpaCy. Затем process() делает из токенов тензор, основываясь на построенном словаре (Torchtext ожидает пакет строк, поэтому необходимо передавать данные в функцию в виде списка спиосков). Затем мы отдаем все это в модель и создаем тензор, на сонове которого делаем предсказание.

In [25]:
def classify_tweet(tweet):
    categories = {0: "Negative", 1:"Positive"}
    processed = TWEET.process([TWEET.preprocess(tweet)])
    processed = processed.to(device)
    return categories[model(processed).argmax().item()]